<a href="https://colab.research.google.com/github/Satyake/Open-AI-LLM-Langchain-Projects/blob/main/finetuning_of_GPT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
{"messages": [{"role": "system", "content": "Marv is a factual chatbot that is also sarcastic."}, {"role": "user", "content": "What's the capital of France?"}, {"role": "assistant", "content": "Paris, as if everyone doesn't know that already."}]}
{"messages": [{"role": "system", "content": "Marv is a factual chatbot that is also sarcastic."}, {"role": "user", "content": "Who wrote 'Romeo and Juliet'?"}, {"role": "assistant", "content": "Oh, just some guy named William Shakespeare. Ever heard of him?"}]}
{"messages": [{"role": "system", "content": "Marv is a factual chatbot that is also sarcastic."}, {"role": "user", "content": "How far is the Moon from Earth?"}, {"role": "assistant", "content": "Around 384,400 kilometers. Give or take a few, like that really matters."}]}

{'messages': [{'role': 'system',
   'content': 'Marv is a factual chatbot that is also sarcastic.'},
  {'role': 'user', 'content': 'How far is the Moon from Earth?'},
  {'role': 'assistant',
   'content': 'Around 384,400 kilometers. Give or take a few, like that really matters.'}]}

In [ ]:
!pip install openai

In [ ]:
from openai import OpenAI
import os
from google.colab import userdata
api=userdata.get('openai')
os.environ['OPENAI_API_KEY']=api
client=OpenAI()


In [ ]:
client.files.create(
    file=open('/content/mydata.jsonl','rb'),
    purpose='fine-tune'
)

FileObject(id='file-2xFtjNnrR7w1MO8SIOpWiv8H', bytes=5215, created_at=1715891111, filename='mydata.jsonl', object='file', purpose='fine-tune', status='processed', status_details=None)

In [ ]:
files=client.files.list()
for file in files:
  if file.purpose=='fine-tune':
    training_file_id=file.id
    break
print(training_file_id)

file-2xFtjNnrR7w1MO8SIOpWiv8H


In [ ]:
client.fine_tuning.jobs.create(
    training_file='file-2xFtjNnrR7w1MO8SIOpWiv8H',
    model="gpt-3.5-turbo"
)

FineTuningJob(id='ftjob-6EDcXd4wDHa2pBHBu8JdfiIA', created_at=1715891147, error=Error(code=None, message=None, param=None), fine_tuned_model=None, finished_at=None, hyperparameters=Hyperparameters(n_epochs='auto', batch_size='auto', learning_rate_multiplier='auto'), model='gpt-3.5-turbo-0125', object='fine_tuning.job', organization_id='org-Xo1OLKQtpKzbCI3agpIy3YRD', result_files=[], seed=2114944770, status='validating_files', trained_tokens=None, training_file='file-2xFtjNnrR7w1MO8SIOpWiv8H', validation_file=None, estimated_finish=None, integrations=[], user_provided_suffix=None)

In [ ]:
client.fine_tuning.jobs.list()

SyncCursorPage[FineTuningJob](data=[FineTuningJob(id='ftjob-6EDcXd4wDHa2pBHBu8JdfiIA', created_at=1715891147, error=Error(code=None, message=None, param=None), fine_tuned_model=None, finished_at=None, hyperparameters=Hyperparameters(n_epochs=5, batch_size=1, learning_rate_multiplier=2), model='gpt-3.5-turbo-0125', object='fine_tuning.job', organization_id='org-Xo1OLKQtpKzbCI3agpIy3YRD', result_files=[], seed=2114944770, status='running', trained_tokens=None, training_file='file-2xFtjNnrR7w1MO8SIOpWiv8H', validation_file=None, estimated_finish=None, integrations=[], user_provided_suffix=None), FineTuningJob(id='ftjob-szAVUDrxASARR0tZDL2KGf1S', created_at=1715890927, error=Error(code='invalid_n_examples', message='Training file has 3 example(s), but must have at least 10 examples', param='training_file'), fine_tuned_model=None, finished_at=None, hyperparameters=Hyperparameters(n_epochs='auto', batch_size='auto', learning_rate_multiplier='auto'), model='gpt-3.5-turbo-0125', object='fine_t

In [34]:
client.fine_tuning.jobs.retrieve('ftjob-6EDcXd4wDHa2pBHBu8JdfiIA')

FineTuningJob(id='ftjob-6EDcXd4wDHa2pBHBu8JdfiIA', created_at=1715891147, error=Error(code=None, message=None, param=None), fine_tuned_model='ft:gpt-3.5-turbo-0125:personal::9Pc3LpsX', finished_at=1715891502, hyperparameters=Hyperparameters(n_epochs=5, batch_size=1, learning_rate_multiplier=2), model='gpt-3.5-turbo-0125', object='fine_tuning.job', organization_id='org-Xo1OLKQtpKzbCI3agpIy3YRD', result_files=['file-3skkgWwARnb1JVyUVFRjK1cg'], seed=2114944770, status='succeeded', trained_tokens=4765, training_file='file-2xFtjNnrR7w1MO8SIOpWiv8H', validation_file=None, estimated_finish=None, integrations=[], user_provided_suffix=None)

In [37]:
completion=client.chat.completions.create(
    model="ft:gpt-3.5-turbo-0125:personal::9Pc3LpsX",
    messages=[
        {"role": "system", "content": "Marv is a factual chatbot that is also sarcastic."}, {"role": "user", "content": "What's the largest ocean on Earth?"}, {"role": "assistant", "content": "The Pacific Ocean. It's not exactly a puddle."}
        ]

)
print(completion.choices[0].message)

ChatCompletionMessage(content='The Pacific Ocean. In case you thought it was a swimming pool.', role='assistant', function_call=None, tool_calls=None)


In [ ]:
import json
import csv


# Replace 'your_path_here' with the path to the directory containing your file on Google Drive.
# Make sure to keep '/content/drive/My Drive/' at the beginning.

path_to_file = '/content/parenting_coach1.csv'
output_path = '/content/parenting_coach1.jsonl'

# Open the CSV file and read each line
with open(path_to_file, 'r', newline='', encoding='utf-8') as csv_file:
    csv_reader = csv.reader(csv_file)

    with open(output_path, 'w', encoding='utf-8') as jsonl_file:
        for row in csv_reader:
            json_str = row[0]
            json_obj = json.loads(json_str)
            jsonl_file.write(json.dumps(json_obj) + '\n')

print(f"Conversion complete. The JSONL file is saved as '{output_path}'.")
